# Atividade 2 - Visão Computacional

O entregável de toda esta atividade vai ser um código-fonte em *Python*. 

Encorajamos vocês a fazerem vídeos demonstrando o resultado e a postar (pode ser privadamente) no YouTube



Você deve ter uma folha com o padrão anexo. 
*Dica:* Se não tiver, é possível fazer também com um tablet ou *smartphone*
 
<img src="folha_atividade.png" width=300>

## Parte 1 - calibração

Ouça a explicação do professor sobre o modelo de câmera *pinhole*  e desenhe a medida $f$ que separa o plano focal da pupila da câmera

Detalhe como calculou $f$

Sabendo o tamanho da linha entre os circulos e o tamanho em pixels da imagem, calcula-se o f utilizando uma distância de controle.

## Parte 2

Modifique um dos exemplos `draw_circles_video.py` ou `videoplay.py` para passar a ler dados da webcam e identificar o círculo magenta e o círculo ciano, usando o `inRange`

## Parte 3

Assumindo que a folha se mantém sempre paralela ao plano de imagem da câmera, imprima a distância entre a folha e sua câmera

## Parte 4

Trace uma linha entre os centros do círculo magenta e do círculo ciano.

Imprima na tela o ângulo entre esta linha e a horizontal

In [3]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
from math import pi
import matplotlib.cm as cm


def find_homography_draw_box(kp1, kp2, img_cena):
    
    out = img_cena.copy()
    
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)


    # Tenta achar uma trasformacao composta de rotacao, translacao e escala que situe uma imagem na outra
    # Esta transformação é chamada de homografia 
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()


    
    h,w = img_original.shape
    # Um retângulo com as dimensões da imagem original
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

    # Transforma os pontos do retângulo para onde estao na imagem destino usando a homografia encontrada
    dst = cv2.perspectiveTransform(pts,M)


    # Desenha um contorno em vermelho ao redor de onde o objeto foi encontrado
    img2b = cv2.polylines(out,[np.int32(dst)],True,(255,255,0),5, cv2.LINE_AA)
    
    return img2b
    
# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

lower = 0
upper = 1

magentaMin = np.array([139,  50,  50], dtype=np.uint8)
magentaMax = np.array([189, 255, 255], dtype=np.uint8)
cianoMin = np.array([85,  50,  50], dtype=np.uint8)
cianoMax = np.array([135, 255, 255], dtype=np.uint8)

#Imagem do logo INSPER
logo_bgr = cv2.imread('insper_logo.png')
logo_gray = cv2.cvtColor(logo_bgr, cv2.COLOR_BGR2GRAY)
original_bgr = logo_bgr
original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)
img_original = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2GRAY)

#Procura features
brisk = cv2.BRISK_create() # Nota: numa versão anterior era a BRISK
kpts = brisk.detect(logo_gray)
x = [k.pt[0] for k in kpts]
y = [k.pt[1] for k in kpts]
# s will correspond to the neighborhood area
s = [(k.size/2)**2 * pi for k in kpts]


#fonte para letra
font = cv2.FONT_HERSHEY_SIMPLEX

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged

# Número mínimo de pontos correspondentes
MIN_MATCH_COUNT = 7

while(True):
    # Capture frame-by-frame
    print("New frame")
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)

    #Faz um map em HSV
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    circles = []

    # reseta o valor da distancia do papel
    papel = None

    # reseta o angulo e vetor para novo frame
    angulo = None
    vetor = ()

    # reset das coordenadas dos centros
    magenta = ()
    ciano = ()

    # Obtains a version of the edges image where we can draw in color
    
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)
    


    # Identificação do logo
    cena_bgr = frame # Imagem do cenario

    # Versões RGB das imagens, para plot
    cena_rgb = cv2.cvtColor(cena_bgr, cv2.COLOR_BGR2RGB)
    
    # Versões grayscale para feature matching
    img_cena = cv2.cvtColor(cena_bgr, cv2.COLOR_BGR2GRAY)

    framed = None

    # Imagem de saída
    out = cena_rgb.copy()

    # Cria o detector BRISK
    brisk = cv2.BRISK_create()

    # Encontra os pontos únicos (keypoints) nas duas imagems
    kp1, des1 = brisk.detectAndCompute(img_original ,None)
    kp2, des2 = brisk.detectAndCompute(img_cena,None)

    # Configura o algoritmo de casamento de features que vê *como* o objeto que deve ser encontrado aparece na imagem
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)

    # Tenta fazer a melhor comparacao usando o algoritmo
    matches = bf.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)


    if len(good)>MIN_MATCH_COUNT:
        # Separa os bons matches na origem e no destino
        print("Matches found")    
        bordas_color = find_homography_draw_box(kp1, kp2, bordas_color)
    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))


    # Versões RGB das imagens, para plot
    original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)
    cena_rgb = cv2.cvtColor(cena_bgr, cv2.COLOR_BGR2RGB)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            #print(i)
            
            mask = cv2.inRange(hsvFrame, magentaMin, magentaMax)
            if mask[i[1]][i[0]] > 200:
                cv2.circle(bordas_color,(i[0],i[1]),i[2],(int(frame[i[1],i[0]][0]), int(frame[i[1],i[0]][1]), int(frame[i[1],i[0]][2])),-1)
                # Salva as coordenadas do circulo magenta
                magenta = (i[0],i[1])           
            mask = cv2.inRange(hsvFrame, cianoMin, cianoMax)
            if mask[i[1]][i[0]] > 200:
                cv2.circle(bordas_color,(i[0],i[1]),i[2],(int(frame[i[1],i[0]][0]), int(frame[i[1],i[0]][1]), int(frame[i[1],i[0]][2])),-1)
                # Salva as coordenadas do círculo ciano
                ciano = (i[0],i[1])

            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)
    
    #Desenha a reta entre os centros dos círculos
    try:
        cv2.line(bordas_color,magenta,ciano,(0,255,0),5)
        if magenta [0] > ciano[0] and magenta[1] > ciano[1]:
            dist = ((magenta[0]-ciano[0])**2+(magenta[1]-ciano[1])**2)**(1/2) # magenta > ciano em x e y
            papel = str(777 * 14 / dist) + "cm"
            angulo = str(np.arctan(abs(magenta[1]-ciano[1])/abs(ciano[0]-magenta[0]))*180/np.pi) + "graus"
            
        elif magenta[0] > ciano[0] and ciano[1] > magenta[1]:
            dist = ((magenta[0]-ciano[0])**2+(ciano[1]-magenta[1])**2)**(1/2)
            papel = str(777 * 14 / dist) + "cm"
            angulo = str(np.arctan(abs(ciano[1]-magenta[1])/abs(magenta[0]-ciano[0]))*180/np.pi) + "graus"

        elif ciano[0] > magenta[0] and ciano[1] > magenta[1]:
            dist = ((ciano[0]-magenta[0])**2+(ciano[1]-magenta[1])**2)**(1/2) # magenta > ciano em x e y
            papel = str(777 * 14 / dist) + "cm"
            angulo = str(180-np.arctan(abs(ciano[1]-magenta[1])/abs(ciano[0]-magenta[0]))*180/np.pi) + "graus"

        elif ciano[0] > magenta[0] and magenta[1] > ciano[1]:
            dist = ((ciano[0]-magenta[0])**2+(magenta[1]-ciano[1])**2)**(1/2)
            papel = str(777 * 14 / dist) + "cm"
            angulo = str(180-np.arctan(abs(magenta[1]-ciano[1])/abs(magenta[0]-ciano[0]))*180/np.pi) + "graus"
            
        elif ciano[1] == magenta[1]:
            angulo = "pi/2"

        else:
            angulo = "0"   

        cv2.putText(bordas_color,papel,(0,130), font, 2,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(bordas_color,angulo,(0,50), font, 2,(255,255,255),2,cv2.LINE_AA)
    except:
        print("cade circulo")

    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
    print("No circles were found")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 1/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found - 1/7
cade circulo
No circles were found
New frame
Not enough matches are found - 0/7
cade circulo
No circles were found
New frame
Not enough matches are found -

In [14]:
print(vetor)

()


## Parte 5

Usando transformada de Hough, desenhe um círculo sobre o círculo ciano e outro sobre o círculo magenta.

**Desafio bônus**: ser capaz de eliminar circulos espúrios (aqueles que não são os da folha)

## Parte 6

Usando `SIFT`, identifique o escrito *Insper* na folha